In [1]:
# !pip install --upgrade -q opensearch-py
# !pip install --upgrade -q requests-aws4auth
# !pip install --upgrade -q boto3
# !pip install --upgrade -q botocore
# !pip install --upgrade -q awscli

In [1]:
import logging
import boto3
import time
import json
import uuid
import pprint
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

In [2]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

[2025-01-31 21:54:00,428] p87239 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

session, region, account_id

(Session(region_name='us-east-1'), 'us-east-1', '174178623257')

In [5]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "bedrock-fm-models-kb-agents"
agent_alias_name = "bedrock-fm-models-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_arn = f"arn:aws:s3:::{bucket_name}"
bedrock_agent_bedrock_allow_policy_name = f"bda-bedrock-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"bda-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"bda-kb-allow-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_bedrock_fm-models'

kb_name = f'bedrock-fm-models-kb-{suffix}'
data_source_name = f'bedrock-fm-models-kb-{suffix}'
kb_files_path = 'kb_documents'
kb_key = 'kb_documents'
kb_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_fm-models'
kb_bedrock_allow_policy_name = f"bd-kb-bedrock-allow-{suffix}"
kb_aoss_allow_policy_name = f"bd-kb-aoss-allow-{suffix}"
kb_s3_allow_policy_name = f"bd-kb-s3-allow-{suffix}"
kb_collection_name = f'bd-kbc-{suffix}'

# Select Amazon titan as the embedding model
embedding_model_arn = f'arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v1'
kb_vector_index_name = "bedrock-knowledge-base-index"
kb_metadataField = 'bedrock-knowledge-base-metadata'
kb_textField = 'bedrock-knowledge-base-text'
kb_vectorField = 'bedrock-knowledge-base-vector'

#### FM Model Seclection

In [6]:
claude_sonnet_3_5 = "anthropic.claude-3-5-sonnet-20241022-v2:0"
nova_pro = "amazon.nova-pro-v1:0"
nova_lite = "amazon.nova-lite-v1:0"
llama_3_3_70b = "meta.llama3-3-70b-instruct-v1:0"

model_id = nova_lite

In [7]:
# agent configuration
agent_instruction = """
You are an agent that support users working with Amazon Bedrock. You have access to documentation about foundational models available in Amazon Bedrock in a Knowledge Base and you can Answer questions from this documentation.
Only answer questions based on the documentation and reply with 
"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" 
If the answer to the question is not available in the documentation
"""

## Upload the dataset to Amazon S3

#### Create the Amazon S3 bucket

In [8]:
s3_client.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'G2QBQ9Q7VX9TNKAD',
  'HostId': 'ybk3G2FgmQdUKND2aIzLet7m7kn1La139P8xmfyHAWrV3GY0JpuLbHrnfmjQnoR9YwIDHo4GKUU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ybk3G2FgmQdUKND2aIzLet7m7kn1La139P8xmfyHAWrV3GY0JpuLbHrnfmjQnoR9YwIDHo4GKUU=',
   'x-amz-request-id': 'G2QBQ9Q7VX9TNKAD',
   'date': 'Fri, 31 Jan 2025 16:24:23 GMT',
   'location': '/bedrock-fm-models-kb-agents-us-east-1-174178623257',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/bedrock-fm-models-kb-agents-us-east-1-174178623257'}

#### Upload dataset to the Amazon S3 bucket

In [9]:
for f in os.listdir(kb_files_path):
    if f.endswith(".pdf"):
        s3_client.upload_file(kb_files_path+'/'+f, bucket_name, kb_key+'/'+f)

## Create a Knowledge Base for Amazon Bedrock

#### Create Knowledge Base Role and Policies

In [10]:
bedrock_kb_allow_fm_model_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                embedding_model_arn
            ]
        }
    ]
}

kb_bedrock_policy_json = json.dumps(bedrock_kb_allow_fm_model_policy_statement)

kb_bedrock_policy = iam_client.create_policy(
    PolicyName=kb_bedrock_allow_policy_name,
    PolicyDocument=kb_bedrock_policy_json
)

In [11]:
# Create IAM policies for KB to access OpenSearch Serverless
bedrock_kb_allow_aoss_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "aoss:APIAccessAll",
            "Resource": [
                f"arn:aws:aoss:{region}:{account_id}:collection/*"
            ]
        }
    ]
}


kb_aoss_policy_json = json.dumps(bedrock_kb_allow_aoss_policy_statement)

kb_aoss_policy = iam_client.create_policy(
    PolicyName=kb_aoss_allow_policy_name,
    PolicyDocument=kb_aoss_policy_json
)

In [12]:
kb_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowKBAccessDocuments",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{bucket_name}/*",
                f"arn:aws:s3:::{bucket_name}"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": f"{account_id}"
                }
            }
        }
    ]
}


kb_s3_json = json.dumps(kb_s3_allow_policy_statement)
kb_s3_policy = iam_client.create_policy(
    PolicyName=kb_s3_allow_policy_name,
    PolicyDocument=kb_s3_json
)

In [13]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
kb_role = iam_client.create_role(
    RoleName=kb_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_aoss_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=kb_role_name,
    PolicyArn=kb_s3_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '5264f7dc-a420-47f0-9bdb-7e9ba8d911cc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 31 Jan 2025 16:24:43 GMT',
   'x-amzn-requestid': '5264f7dc-a420-47f0-9bdb-7e9ba8d911cc',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [14]:
kb_role_arn = kb_role["Role"]["Arn"]
kb_role_arn

'arn:aws:iam::174178623257:role/AmazonBedrockExecutionRoleForKnowledgeBase_bedrock_fm-models'

#### Create Vector Database

In [15]:
# Create OpenSearch Collection
security_policy_json = {
    "Rules": [
        {
            "ResourceType": "collection",
            "Resource":[
                f"collection/{kb_collection_name}"
            ]
        }
    ],
    "AWSOwnedKey": True
}
security_policy = open_search_serverless_client.create_security_policy(
    description='security policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(security_policy_json),
    type='encryption'
)

In [16]:
network_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "dashboard"
      },
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "ResourceType": "collection"
      }
    ],
    "AllowFromPublic": True
  }
]

network_policy = open_search_serverless_client.create_security_policy(
    description='network policy of aoss collection',
    name=kb_collection_name,
    policy=json.dumps(network_policy_json),
    type='network'
)

In [17]:
response = sts_client.get_caller_identity()
current_role = response['Arn']
current_role

'arn:aws:iam::174178623257:user/Siddharth'

In [18]:
data_policy_json = [
  {
    "Rules": [
      {
        "Resource": [
          f"collection/{kb_collection_name}"
        ],
        "Permission": [
          "aoss:DescribeCollectionItems",
          "aoss:CreateCollectionItems",
          "aoss:UpdateCollectionItems",
          "aoss:DeleteCollectionItems"
        ],
        "ResourceType": "collection"
      },
      {
        "Resource": [
          f"index/{kb_collection_name}/*"
        ],
        "Permission": [
            "aoss:CreateIndex",
            "aoss:DeleteIndex",
            "aoss:UpdateIndex",
            "aoss:DescribeIndex",
            "aoss:ReadDocument",
            "aoss:WriteDocument"
        ],
        "ResourceType": "index"
      }
    ],
    "Principal": [
        kb_role_arn,
        f"arn:aws:sts::{account_id}:assumed-role/Admin/*",
        current_role
    ],
    "Description": ""
  }
]

data_policy = open_search_serverless_client.create_access_policy(
    description='data access policy for aoss collection',
    name=kb_collection_name,
    policy=json.dumps(data_policy_json),
    type='data'
)

In [19]:
opensearch_collection_response = open_search_serverless_client.create_collection(
    description='OpenSearch collection for Amazon Bedrock Knowledge Base',
    name=kb_collection_name,
    standbyReplicas='DISABLED',
    type='VECTORSEARCH'
)

opensearch_collection_response

{'createCollectionDetail': {'arn': 'arn:aws:aoss:us-east-1:174178623257:collection/0cana936kf6yyrud00c2',
  'createdDate': 1738340691991,
  'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base',
  'id': '0cana936kf6yyrud00c2',
  'kmsKeyArn': 'auto',
  'lastModifiedDate': 1738340691991,
  'name': 'bd-kbc-us-east-1-174178623257',
  'standbyReplicas': 'DISABLED',
  'status': 'CREATING',
  'type': 'VECTORSEARCH'},
 'ResponseMetadata': {'RequestId': 'a88230e6-27b0-4dac-87cc-5380dadeebd9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a88230e6-27b0-4dac-87cc-5380dadeebd9',
   'date': 'Fri, 31 Jan 2025 16:24:52 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '394',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [20]:
collection_arn = opensearch_collection_response["createCollectionDetail"]["arn"]
collection_arn

'arn:aws:aoss:us-east-1:174178623257:collection/0cana936kf6yyrud00c2'

In [21]:
# wait for collection creation
response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
# Periodically check collection status
while (response['collectionDetails'][0]['status']) == 'CREATING':
    print('Creating collection...')
    time.sleep(30)
    response = open_search_serverless_client.batch_get_collection(names=[kb_collection_name])
print('\nCollection successfully created:')
print(response["collectionDetails"])
# Extract the collection endpoint from the response
host = (response['collectionDetails'][0]['collectionEndpoint'])
final_host = host.replace("https://", "")
final_host

Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...
Creating collection...

Collection successfully created:
[{'arn': 'arn:aws:aoss:us-east-1:174178623257:collection/0cana936kf6yyrud00c2', 'collectionEndpoint': 'https://0cana936kf6yyrud00c2.us-east-1.aoss.amazonaws.com', 'createdDate': 1738340691991, 'dashboardEndpoint': 'https://0cana936kf6yyrud00c2.us-east-1.aoss.amazonaws.com/_dashboards', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base', 'id': '0cana936kf6yyrud00c2', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1738340928374, 'name': 'bd-kbc-us-east-1-174178623257', 'standbyReplicas': 'DISABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}]


'0cana936kf6yyrud00c2.us-east-1.aoss.amazonaws.com'

#### Create OpenSearch Index

In [22]:
credentials = boto3.Session().get_credentials()
service = 'aoss'
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key,
    region, 
    service, 
    session_token=credentials.token
)

# Build the OpenSearch client
open_search_client = OpenSearch(
    hosts=[{'host': final_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)
# It can take up to a minute for data access rules to be enforced
time.sleep(45)
index_body = {
    "settings": {
        "index.knn": True,
        "number_of_shards": 1,
        "knn.algo_param.ef_search": 512,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {}
    }
}

index_body["mappings"]["properties"][kb_vectorField] = {
    "type": "knn_vector",
    "dimension": 1536,
    "method": {
         "name": "hnsw",
         "engine": "faiss"
    },
}

index_body["mappings"]["properties"][kb_textField] = {
    "type": "text"
}

index_body["mappings"]["properties"][kb_metadataField] = {
    "type": "text"
}

# Create index
response = open_search_client.indices.create(kb_vector_index_name, body=index_body)
print('\nCreating index:')
print(response)

[2025-01-31 21:58:57,438] p87239 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2025-01-31 21:59:46,179] p87239 {base.py:258} INFO - PUT https://0cana936kf6yyrud00c2.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-index [status:200 request:3.693s]



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'bedrock-knowledge-base-index'}


#### Create Knowledge Base

In [23]:
storage_configuration = {
    'opensearchServerlessConfiguration': {
        'collectionArn': collection_arn, 
        'fieldMapping': {
            'metadataField': kb_metadataField,
            'textField': kb_textField,
            'vectorField': kb_vectorField
        },
        'vectorIndexName': kb_vector_index_name
    },
    'type': 'OPENSEARCH_SERVERLESS'
}

In [ ]:
delete KB from bedrock 

In [24]:
# Creating the knowledge base
try:
    # ensure the index is created and available
    time.sleep(45)
    kb_obj = bedrock_agent_client.create_knowledge_base(
        name=kb_name, 
        description='KB that contains the bedrock documentation',
        roleArn=kb_role_arn,
        knowledgeBaseConfiguration={
            'type': 'VECTOR',  # Corrected type
            'vectorKnowledgeBaseConfiguration': {
                'embeddingModelArn': embedding_model_arn
            }
        },
        storageConfiguration=storage_configuration
    )

    # Pretty print the response
    pprint.pprint(kb_obj)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1023',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 31 Jan 2025 16:30:43 GMT',
                                      'x-amz-apigw-id': 'FQwbuEmQoAMEQLw=',
                                      'x-amzn-requestid': 'f38d2041-5101-44bf-918b-581f1ed7c6de',
                                      'x-amzn-trace-id': 'Root=1-679cfab1-4e20e9e409a6f038333af601'},
                      'HTTPStatusCode': 202,
                      'RequestId': 'f38d2041-5101-44bf-918b-581f1ed7c6de',
                      'RetryAttempts': 0},
 'knowledgeBase': {'createdAt': datetime.datetime(2025, 1, 31, 16, 30, 41, 177972, tzinfo=tzutc()),
                   'description': 'KB that contains the bedrock documentation',
                   'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:174178623257:knowledge-base/FC57

In [25]:
# Define the S3 configuration for your data source
s3_configuration = {
    'bucketArn': bucket_arn,
    'inclusionPrefixes': [kb_key]  
}

# Define the data source configuration
data_source_configuration = {
    's3Configuration': s3_configuration,
    'type': 'S3'
}

knowledge_base_id = kb_obj["knowledgeBase"]["knowledgeBaseId"]
knowledge_base_arn = kb_obj["knowledgeBase"]["knowledgeBaseArn"]

chunking_strategy_configuration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# Create the data source
try:
    # ensure that the KB is created and available
    time.sleep(45)
    data_source_response = bedrock_agent_client.create_data_source(
        knowledgeBaseId=knowledge_base_id,
        name=data_source_name,
        description='DataSource for the bedrock documentation',
        dataSourceConfiguration=data_source_configuration,
        vectorIngestionConfiguration = {
            "chunkingConfiguration": chunking_strategy_configuration
        }
    )

    # Pretty print the response
    pprint.pprint(data_source_response)

except Exception as e:
    print(f"Error occurred: {e}")

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '666',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 31 Jan 2025 16:31:28 GMT',
                                      'x-amz-apigw-id': 'FQwjLHlbIAMEn-A=',
                                      'x-amzn-requestid': '26480f56-200c-474e-8aa1-b34dbf07e003',
                                      'x-amzn-trace-id': 'Root=1-679cfae0-2d240662625affe6258cfafb'},
                      'HTTPStatusCode': 200,
                      'RequestId': '26480f56-200c-474e-8aa1-b34dbf07e003',
                      'RetryAttempts': 0},
 'dataSource': {'createdAt': datetime.datetime(2025, 1, 31, 16, 31, 28, 824435, tzinfo=tzutc()),
                'dataDeletionPolicy': 'DELETE',
                'dataSourceConfiguration': {'s3Configuration': {'bucketArn': 'arn:aws:s3:::bedrock-fm-models-kb-agents-us-east-1-17417

#### Start ingestion job

In [26]:
# Start an ingestion job
data_source_id = data_source_response["dataSource"]["dataSourceId"]
start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=knowledge_base_id, 
    dataSourceId=data_source_id
)

## Create Agent

#### Create Agent IAM role and policies

In [27]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [28]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)
agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)

In [29]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)


iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'ac6ae0ac-f633-473c-a499-820c87127750',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 31 Jan 2025 16:32:04 GMT',
   'x-amzn-requestid': 'ac6ae0ac-f633-473c-a499-820c87127750',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Create Agent

In [30]:
# Create Agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent supporting Amazon Bedrock Developers.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

In [31]:
agent_id = response['agent']['agentId']
agent_id

'EMVMMJYNXP'

#### Associate agent to the Knowledge Base

In [32]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {kb_name} knowledge base to provide accurate responses to the questions about Foundation Models in Amazon Bedrock.',
    knowledgeBaseId=knowledge_base_id 
)

#### Prepare Agent

In [33]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '49f81366-1e6c-4333-8d88-960f2ee37eec',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 31 Jan 2025 16:32:11 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '49f81366-1e6c-4333-8d88-960f2ee37eec',
   'x-amz-apigw-id': 'FQwp3GZ3oAMElWg=',
   'x-amzn-trace-id': 'Root=1-679cfb0b-31871d9a1df12858441d3f88'},
  'RetryAttempts': 0},
 'agentId': 'EMVMMJYNXP',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 1, 31, 16, 32, 11, 855567, tzinfo=tzutc())}

## Testing Agent

In [34]:
# Pause to make sure agent is prepared
time.sleep(30)

agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

# Pause to make sure agent alias is ready
time.sleep(30)

In [35]:
agent_alias

{'ResponseMetadata': {'RequestId': '44e277d8-bec6-42ed-816e-b83b9aeffdbd',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 31 Jan 2025 16:32:44 GMT',
   'content-type': 'application/json',
   'content-length': '349',
   'connection': 'keep-alive',
   'x-amzn-requestid': '44e277d8-bec6-42ed-816e-b83b9aeffdbd',
   'x-amz-apigw-id': 'FQwu7GClIAMEOLw=',
   'x-amzn-trace-id': 'Root=1-679cfb2b-5123aa672d141a45558c9980'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:174178623257:agent-alias/EMVMMJYNXP/ELLLFES6F8',
  'agentAliasId': 'ELLLFES6F8',
  'agentAliasName': 'bedrock-fm-models-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'EMVMMJYNXP',
  'createdAt': datetime.datetime(2025, 1, 31, 16, 32, 44, 53863, tzinfo=tzutc()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2025, 1, 31, 16, 32, 44, 53863, tzinfo=tzutc())}}

In [36]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="What are the models available on Bedrock?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2025-01-31 22:03:15,476] p87239 {3358150927.py:19} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Fri, 31 Jan 2025 16:33:15 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '114a456a-dff1-11ef-bec8-da5cbc4eb4cc',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '99142cef-a12b-4aa6-b5e8-ea3417fba36c'},
                      'HTTPStatusCode': 200,
                      'RequestId': '99142cef-a12b-4aa6-b5e8-ea3417fba36c',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x1119bbd90>,
 'contentType': 'application/json',
 'sessionId': '114a456a-dff1-11ef-bec8-da5cbc4eb4cc'}


In [37]:
%%time
event_stream = agentResponse['completion']
traces = []


try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
            traces.append(event['trace'])
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2025-01-31 22:03:15,642] p87239 {<timed exec>:14} INFO - {
  "agentAliasId": "ELLLFES6F8",
  "agentId": "EMVMMJYNXP",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/EMVMMJYNXP/ELLLFES6F8"
    }
  ],
  "sessionId": "114a456a-dff1-11ef-bec8-da5cbc4eb4cc",
  "trace": {
    "orchestrationTrace": {
      "invocationInput": {
        "invocationType": "KNOWLEDGE_BASE",
        "knowledgeBaseLookupInput": {
          "knowledgeBaseId": "FC57MFDQHW",
          "text": "What are the models available on Bedrock?"
        },
        "traceId": "99142cef-a12b-4aa6-b5e8-ea3417fba36c-0"
      }
    }
  }
}
[2025-01-31 22:03:16,111] p87239 {<timed exec>:14} INFO - {
  "agentAliasId": "ELLLFES6F8",
  "agentId": "EMVMMJYNXP",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:174178623257:agent-alias/EMVMMJYNXP/ELLLFES6F8"
    }
  ],
  "sessionId": "114a456a-dff1-11ef-bec8-da5cbc4

CPU times: user 9.17 ms, sys: 4.84 ms, total: 14 ms
Wall time: 7.36 s


In [38]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

The models available on Bedrock include:

    
        AI21 Labs has developed the Jurassic-2 models, known for their strong capabilities in natural language processing. These models are designed to handle a wide array of tasks such as text generation, summarization, and question answering. Jurassic-2 emphasizes user customization, allowing businesses to fine-tune the model for specific applications[1]. Cohere models focus on providing robust natural language processing capabilities tailored for enterprise needs. These models are optimized for tasks such as semantic search, classification, and text generation. Cohere emphasizes ease of integration into existing workflows and systems, making it accessible for businesses looking to leverage AI without extensive technical expertise. The models also support customization options that allow organizations to train the model on their specific datasets for improved performance in niche applications[1]. Meta's Llama models represent a new front

In [39]:
traces

[{'agentAliasId': 'ELLLFES6F8',
  'agentId': 'EMVMMJYNXP',
  'agentVersion': '1',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:174178623257:agent-alias/EMVMMJYNXP/ELLLFES6F8'}],
  'sessionId': '114a456a-dff1-11ef-bec8-da5cbc4eb4cc',
  'trace': {'orchestrationTrace': {'invocationInput': {'invocationType': 'KNOWLEDGE_BASE',
     'knowledgeBaseLookupInput': {'knowledgeBaseId': 'FC57MFDQHW',
      'text': 'What are the models available on Bedrock?'},
     'traceId': '99142cef-a12b-4aa6-b5e8-ea3417fba36c-0'}}}},
 {'agentAliasId': 'ELLLFES6F8',
  'agentId': 'EMVMMJYNXP',
  'agentVersion': '1',
  'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-east-1:174178623257:agent-alias/EMVMMJYNXP/ELLLFES6F8'}],
  'sessionId': '114a456a-dff1-11ef-bec8-da5cbc4eb4cc',
  'trace': {'orchestrationTrace': {'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "Amazon Bedrock Foundation Models Amazon Bedrock is a fully managed service provided by AWS 

## Analyzing the Trace

In [41]:
for trace in traces:
    print(trace['trace'])
    print()

{'orchestrationTrace': {'invocationInput': {'invocationType': 'KNOWLEDGE_BASE', 'knowledgeBaseLookupInput': {'knowledgeBaseId': 'FC57MFDQHW', 'text': 'What are the models available on Bedrock?'}, 'traceId': '99142cef-a12b-4aa6-b5e8-ea3417fba36c-0'}}}

{'orchestrationTrace': {'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "Amazon Bedrock Foundation Models Amazon Bedrock is a fully managed service provided by AWS that offers access to a diverse range of high-performing foundation models (FMs) from leading AI companies. This service enables businesses to leverage advanced generative AI capabilities through a unified API, making it easier to experiment with and deploy models tailored to specific use cases. With Amazon Bedrock, users can customize these models with their data, ensuring that the AI solutions are relevant and aligned with their operational needs. The foundation models supported by Amazon Bedrock include offerings from various provid

##### idx 0

In [42]:
traces[0]['trace']['orchestrationTrace']

{'invocationInput': {'invocationType': 'KNOWLEDGE_BASE',
  'knowledgeBaseLookupInput': {'knowledgeBaseId': 'FC57MFDQHW',
   'text': 'What are the models available on Bedrock?'},
  'traceId': '99142cef-a12b-4aa6-b5e8-ea3417fba36c-0'}}

##### idx 1

In [43]:
traces[1]['trace']['orchestrationTrace']

{'observation': {'knowledgeBaseLookupOutput': {'retrievedReferences': [{'content': {'text': "Amazon Bedrock Foundation Models Amazon Bedrock is a fully managed service provided by AWS that offers access to a diverse range of high-performing foundation models (FMs) from leading AI companies. This service enables businesses to leverage advanced generative AI capabilities through a unified API, making it easier to experiment with and deploy models tailored to specific use cases. With Amazon Bedrock, users can customize these models with their data, ensuring that the AI solutions are relevant and aligned with their operational needs. The foundation models supported by Amazon Bedrock include offerings from various providers such as AI21 Labs, Anthropic, Cohere, Meta, and Mistral AI. Each model is designed for different applications, ranging from text generation and summarization to image processing and semantic search. This variety allows organizations to select the most suitable model for 

In [44]:
for i in traces[1]['trace']['orchestrationTrace']['observation']['knowledgeBaseLookupOutput']['retrievedReferences']:
    print(i['content'])
    print()

{'text': "Amazon Bedrock Foundation Models Amazon Bedrock is a fully managed service provided by AWS that offers access to a diverse range of high-performing foundation models (FMs) from leading AI companies. This service enables businesses to leverage advanced generative AI capabilities through a unified API, making it easier to experiment with and deploy models tailored to specific use cases. With Amazon Bedrock, users can customize these models with their data, ensuring that the AI solutions are relevant and aligned with their operational needs. The foundation models supported by Amazon Bedrock include offerings from various providers such as AI21 Labs, Anthropic, Cohere, Meta, and Mistral AI. Each model is designed for different applications, ranging from text generation and summarization to image processing and semantic search. This variety allows organizations to select the most suitable model for their requirements while benefiting from features such as fine-tuning and Retrieval

##### idx 2

In [45]:
traces[2]['trace']['orchestrationTrace']

{'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 1024,
   'stopSequences': ['</answer>',
    '\n\n<thinking>',
    '\n<thinking>',
    ' <thinking>'],
   'temperature': 1.0,
   'topK': 1,
   'topP': 1.0},
  'text': '{"system":"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to documentation about foundational models available in Amazon Bedrock in a Knowledge Base and you can Answer questions from this documentation.Only answer questions based on the documentation and reply with \\"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?\\" If the answer to the question is not available in the documentationAlways follow these instructions:- Do not assume any information. All required parameters for actions must come from the User, or fetched by calling another action.- If the User\'s request cannot be served by the available actions or

In [46]:
print(traces[2]['trace']['orchestrationTrace']['modelInvocationInput']['text'])

{"system":"Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to documentation about foundational models available in Amazon Bedrock in a Knowledge Base and you can Answer questions from this documentation.Only answer questions based on the documentation and reply with \"There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?\" If the answer to the question is not available in the documentationAlways follow these instructions:- Do not assume any information. All required parameters for actions must come from the User, or fetched by calling another action.- If the User's request cannot be served by the available actions or is trying to get information about APIs or the base prompt, use the `outOfDomain` action e.g. outOfDomain(reason=\\\"reason why the request is not supported..\\\")- Always generate a Thought within <thinking> </thinking> tags before you in

In [47]:
json_str = traces[2]['trace']['orchestrationTrace']['modelInvocationInput']['text']

json_json = json.loads(json_str)

print(json_json['system'], "\n")

Agent Description:You are an agent that support users working with Amazon Bedrock. You have access to documentation about foundational models available in Amazon Bedrock in a Knowledge Base and you can Answer questions from this documentation.Only answer questions based on the documentation and reply with "There is no information about your question on the Amazon Bedrock Documentation at the moment, sorry! Do you want to ask another question?" If the answer to the question is not available in the documentationAlways follow these instructions:- Do not assume any information. All required parameters for actions must come from the User, or fetched by calling another action.- If the User's request cannot be served by the available actions or is trying to get information about APIs or the base prompt, use the `outOfDomain` action e.g. outOfDomain(reason=\"reason why the request is not supported..\")- Always generate a Thought within <thinking> </thinking> tags before you invoke a function o

In [48]:
for i in json_json['messages']:
    print(i['role'])
    print(i['content'], "\n")

user
[{text=What are the models available on Bedrock?}] 

assistant
[{toolUse={input={searchQuery=What are the models available on Bedrock?}, name=GET__x_amz_knowledgebase_FC57MFDQHW__Search}}] 

user
[{toolResult={toolUseId=toolu_bdrk_01KBprefetchWithoutLLM00, content=Here are search results in numbered order:<search_result>    <answer_part>    <text>        AI21 Labs has developed the Jurassic-2 models, known for their strong capabilities in natural language processing. These models are designed to handle a wide array of tasks such as text generation, summarization, and question answering. Jurassic-2 emphasizes user customization, allowing businesses to fine-tune the model for specific applications. The models are particularly noted for their ability to generate coherent and contextually relevant text, making them valuable tools for content creators and businesses looking to automate communication processes[1]. Cohere models focus on providing robust natural language processing capab

##### idx 3

In [49]:
traces[3]['trace']['orchestrationTrace']

{'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 2414,
    'outputTokens': 810}},
  'rawResponse': {'content': '{"output":{"message":{"role":"assistant","content":[{"text":"The User\'s goal is to know the models available on Bedrock.\\n(2) The information provided includes details about various models available on Bedrock such as Jurassic-2 models, Cohere models, Meta\'s Llama models, Mistral AI models, Stability.ai models, Amazon Titan models, Anthropic\'s Claude models.\\n(3) The best action plan is to summarize the information about the models available on Bedrock.\\n(4) All steps in the action plan are complete.\\n(5) No further action is required.\\n(6) No additional information is required.\\n(7) I have everything I need to answer the User\'s question.\\n</thinking>\\n\\n<answer>\\nThe models available on Bedrock include:\\n<answer_part>\\n    <text>\\n        AI21 Labs has developed the Jurassic-2 models, known for their strong capabilities in natural language pr

In [50]:
json_str = traces[3]['trace']['orchestrationTrace']['modelInvocationOutput']['rawResponse']['content']

json_json = json.loads(json_str)

json_json['output']['message']['role']


print(json_json['output']['message']['content'][0]['text'])

The User's goal is to know the models available on Bedrock.
(2) The information provided includes details about various models available on Bedrock such as Jurassic-2 models, Cohere models, Meta's Llama models, Mistral AI models, Stability.ai models, Amazon Titan models, Anthropic's Claude models.
(3) The best action plan is to summarize the information about the models available on Bedrock.
(4) All steps in the action plan are complete.
(5) No further action is required.
(6) No additional information is required.
(7) I have everything I need to answer the User's question.
</thinking>

<answer>
The models available on Bedrock include:
<answer_part>
    <text>
        AI21 Labs has developed the Jurassic-2 models, known for their strong capabilities in natural language processing. These models are designed to handle a wide array of tasks such as text generation, summarization, and question answering. Jurassic-2 emphasizes user customization, allowing businesses to fine-tune the model fo

##### idx 4

In [51]:
traces[4]['trace']['orchestrationTrace']

{'rationale': {'text': "The User's goal is to know the models available on Bedrock.\n(2) The information provided includes details about various models available on Bedrock such as Jurassic-2 models, Cohere models, Meta's Llama models, Mistral AI models, Stability.ai models, Amazon Titan models, Anthropic's Claude models.\n(3) The best action plan is to summarize the information about the models available on Bedrock.\n(4) All steps in the action plan are complete.\n(5) No further action is required.\n(6) No additional information is required.\n(7) I have everything I need to answer the User's question.",
  'traceId': '99142cef-a12b-4aa6-b5e8-ea3417fba36c-1'}}

In [52]:
print(traces[4]['trace']['orchestrationTrace']['rationale']['text'])

The User's goal is to know the models available on Bedrock.
(2) The information provided includes details about various models available on Bedrock such as Jurassic-2 models, Cohere models, Meta's Llama models, Mistral AI models, Stability.ai models, Amazon Titan models, Anthropic's Claude models.
(3) The best action plan is to summarize the information about the models available on Bedrock.
(4) All steps in the action plan are complete.
(5) No further action is required.
(6) No additional information is required.
(7) I have everything I need to answer the User's question.


##### idx 5

In [53]:
traces[5]['trace']['orchestrationTrace']

{'observation': {'finalResponse': {'text': "The models available on Bedrock include:\n\n    \n        AI21 Labs has developed the Jurassic-2 models, known for their strong capabilities in natural language processing. These models are designed to handle a wide array of tasks such as text generation, summarization, and question answering. Jurassic-2 emphasizes user customization, allowing businesses to fine-tune the model for specific applications[1]. Cohere models focus on providing robust natural language processing capabilities tailored for enterprise needs. These models are optimized for tasks such as semantic search, classification, and text generation. Cohere emphasizes ease of integration into existing workflows and systems, making it accessible for businesses looking to leverage AI without extensive technical expertise. The models also support customization options that allow organizations to train the model on their specific datasets for improved performance in niche application

In [54]:
print(traces[5]['trace']['orchestrationTrace']['observation']['finalResponse']['text'])

The models available on Bedrock include:

    
        AI21 Labs has developed the Jurassic-2 models, known for their strong capabilities in natural language processing. These models are designed to handle a wide array of tasks such as text generation, summarization, and question answering. Jurassic-2 emphasizes user customization, allowing businesses to fine-tune the model for specific applications[1]. Cohere models focus on providing robust natural language processing capabilities tailored for enterprise needs. These models are optimized for tasks such as semantic search, classification, and text generation. Cohere emphasizes ease of integration into existing workflows and systems, making it accessible for businesses looking to leverage AI without extensive technical expertise. The models also support customization options that allow organizations to train the model on their specific datasets for improved performance in niche applications[1]. Meta's Llama models represent a new front

#### Error in Sonnet

When using Claude Sonnet 3.5 the following error:

```text
EventStreamError: An error occurred (validationException) when calling the InvokeAgent operation: Invocation of model ID anthropic.claude-3-5-sonnet-20241022-v2:0 with on-demand throughput isn’t supported. Retry your request with the ID or ARN of an inference profile that contains this model.

During handling of the above exception, another exception occurred:

Exception                                 Traceback (most recent call last)
File <timed exec>:15

Exception: ('unexpected event.', EventStreamError('An error occurred (validationException) when calling the InvokeAgent operation: Invocation of model ID anthropic.claude-3-5-sonnet-20241022-v2:0 with on-demand throughput isn’t supported. Retry your request with the ID or ARN of an inference profile that contains this model.'))
```

## CleanUp

In [56]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [57]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [58]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'ZVENXWQD5A2TVJ0R',
  'HostId': 'wCAV9WWAZJdF8CYte3IpXl4U3VpKkbcgUpBY2FZFuSWNASJf3WyAXSkbauGriXhJXk2eF3vu9cc=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'wCAV9WWAZJdF8CYte3IpXl4U3VpKkbcgUpBY2FZFuSWNASJf3WyAXSkbauGriXhJXk2eF3vu9cc=',
   'x-amz-request-id': 'ZVENXWQD5A2TVJ0R',
   'date': 'Fri, 31 Jan 2025 16:33:59 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [59]:
# Delete IAM Roles and policies and Knowledge Base files
for policy in [
    agent_bedrock_policy, 
    agent_kb_schema_policy,
    kb_bedrock_policy,
    kb_aoss_policy,
    kb_s3_policy
]:
    response = iam_client.list_entities_for_policy(
        PolicyArn=policy['Policy']['Arn'],
        EntityFilter='Role'
    )

    for role in response['PolicyRoles']:
        iam_client.detach_role_policy(
            RoleName=role['RoleName'], 
            PolicyArn=policy['Policy']['Arn']
        )

    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
    )

In [60]:
for role_name in [
    agent_role_name, 
    kb_role_name
]:
    try: 
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as e:
        print(e)
        print("couldn't delete role", role_name)
        
    
try:

    open_search_serverless_client.delete_collection(
        id=opensearch_collection_response["createCollectionDetail"]["id"]
    )

    open_search_serverless_client.delete_access_policy(
          name=kb_collection_name,
          type='data'
    )    

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='network'
    )   

    open_search_serverless_client.delete_security_policy(
          name=kb_collection_name,
          type='encryption'
    )    
    bedrock_agent_client.delete_knowledge_base(
        knowledgeBaseId=knowledge_base_id
    )
except Exception as e:
    print(e)
    print()